# Multi Year score
Given a target year, it computes a score for each image year of adquisition

This score is a constant value for the whole scene (not per-pixel)

In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import ui, tools, collection

In [3]:
from geebap import scores
from geebap.season import Season

In [4]:
p = ee.Geometry.Point([-71.33972167968751, -42.737619925503054])
aoi = p.buffer(8000).bounds()

In [5]:
col = collection.Landsat8SR()

In [6]:
colEE = col.collection.filterBounds(aoi).map(lambda img: img.addBands(col.ndvi(img)))

In [7]:
start = '2015-11-15'
end = '2019-02-15'

In [8]:
colEE = colEE.filterDate(start, end)

In [9]:
year = 2018

In [10]:
years = list(range(2016, 2020))

## Add YEAR_BAP property

In [11]:
season = Season('11-15', '02-15')

In [12]:
ui.eprint(colEE.size())

200



In [13]:
date_range_first = season.add_year(years[0])
first_col = colEE.filterDate(date_range_first.start(), date_range_first.end())
first_col = first_col.map(lambda img: img.set('YEAR_BAP', years[0]))
collections = ee.List([first_col.toList(first_col.size())])

rest = ee.List(years[1:])

def wrap(year, thelist):
    thelist = ee.List(thelist)
    year = ee.Number(year)
    date_range = season.add_year(year)
    filtered = colEE.filterDate(date_range.start(), date_range.end())
    # add YEAR_BAP
    filt = filtered.map(lambda img: img.set('YEAR_BAP', year))
    filt_list = filt.toList(filt.size())
    return thelist.add(filt_list)

final_col = ee.List(rest.iterate(wrap, collections))

In [14]:
final_col = ee.ImageCollection.fromImages(final_col.flatten())

## Score Linear

In [15]:
score = scores.MultiYear(season=season, main_year=year)

In [16]:
newcol = score.apply(final_col, target_year=year, year_property='YEAR_BAP')

In [17]:
chart = ui.chart.Image.series(newcol, p, scale=30, bands=['year_score'], label_bands=['linear'])

## Score Gauss

In [18]:
newcol_n = score.apply(final_col, target_year=year, function='gauss')

In [19]:
chart_n = ui.chart.Image.series(newcol_n, p, scale=30, bands=['year_score'], label_bands=['gauss stretch 1'])

## PLOT

In [20]:
both = chart.cat(chart_n)
both.renderWidget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [46]:
# both.dataframe

## Using collection

In [21]:
newcol_map = score.map(final_col)

In [22]:
chart = ui.chart.Image.series(newcol_map, p, scale=30, bands=['score-multi'], label_bands=['linear'])

In [23]:
chart.renderWidget(width='800px')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…